In [37]:
import sys
sys.path.append('..')

In [38]:
import lasp.differential

import scipy.signal
import numpy

In [39]:
arr = numpy.reshape(numpy.arange(0, 9), (3, 3))
arr

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [45]:
# Derivation with circular boundary

## With numpy
dx1 = lasp.differential.dx(arr)

## With convolution (ie spatial domain)
dx2 = scipy.signal.convolve2d(arr, numpy.array([[1, -1]]), mode='same', boundary='wrap')

## With point wise product
### Spatial convolution becomes a point wise product in frequency domain
### For apply a filter in frequency domain, you must padded filter with
### even dimensions that image then you make point wise product in frequency doamin
dx_oper = numpy.array([1, -1]) # operator
dx_oper_pad = numpy.zeros(shape=(3, 3))
dx_oper_pad[0, 0:2] = dx_oper # operator padded
dx_oper_pad_fft = numpy.fft.fft2(dx_oper_pad) # operator in frequency domain
arr_fft = numpy.fft.fft2(arr) # img in frequency domain
dx_fft = dx_oper_pad_fft * arr_fft # Point wise product in frequency domain
dx3 = numpy.real(numpy.fft.ifft2(dx_fft))

## BCCB tricks
DX_diag = lasp.utils.fourier_diagonalization(numpy.arra, shape_out=DX.shape)
print(numpy.real(numpy.fft.ifft2(DX_diag * numpy.fft.fft2(arr))))




print(dx1)
print(dx2)
print(dx3)

[[-2.  1.  1.]
 [-2.  1.  1.]
 [-2.  1.  1.]]
[[-2  1  1]
 [-2  1  1]
 [-2  1  1]]
[[-2.  1.  1.]
 [-2.  1.  1.]
 [-2.  1.  1.]]


In [41]:
dy1 = lasp.differential.dy(arr)
dy2 = scipy.signal.convolve2d(arr, numpy.array([[1], [-1]]), mode='same', boundary='wrap')
print(dy1)
print(dy2)

[[-6. -6. -6.]
 [ 3.  3.  3.]
 [ 3.  3.  3.]]
[[-6 -6 -6]
 [ 3  3  3]
 [ 3  3  3]]


In [42]:
import lasp.utils


[[ 1. -1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
[[-2.  1.  1.]
 [-2.  1.  1.]
 [-2.  1.  1.]]


In [44]:
dxt = lasp.differential.dxT(arr)
print(dxt)
print(numpy.real(numpy.fft.ifft2(numpy.conj(numpy.fft.fft2(DX)) * numpy.fft.fft2(arr))))

[[-1. -1.  2.]
 [-1. -1.  2.]
 [-1. -1.  2.]]
[[-1. -1.  2.]
 [-1. -1.  2.]
 [-1. -1.  2.]]


In [31]:
DY = numpy.zeros(shape=(512, 512))
DY[0:2, 0] = numpy.transpose(numpy.array([1, -1]))
print(DY)
# print(numpy.real(numpy.fft.ifft2(numpy.fft.fft2(DY) * numpy.fft.fft2(arr))))

[[ 1.  0.  0. ...  0.  0.  0.]
 [-1.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]


In [36]:
# FDX = numpy.fft.fft2(DX)
# FDY = numpy.fft.fft2(DY)
# FDX2 = numpy.abs(FDX)**2
# FDY2 = numpy.abs(FDY)**2
# FDX2+FDY2

In [35]:


# DY_diag = lasp.utils.fourier_diagonalization(DY, shape_out=DY.shape)
# print(DX_diag)
# print(DY_diag)

[[ 1.  1. -2.]
 [ 1.  1. -2.]
 [ 1.  1. -2.]]


In [21]:
DX_diag_2 = numpy.abs(DX_diag)**2
DY_diag_2 = numpy.abs(DY_diag)**2

In [22]:
lap = DX_diag_2 + DY_diag_2
lap

array([[0., 3., 3.],
       [3., 6., 6.],
       [3., 6., 6.]])

In [23]:
DY = numpy.zeros_like(arr)
DY[0, 0] = -1
DY[0, 2] = 1
print(DY)
print(numpy.real(numpy.fft.ifft2(numpy.fft.fft2(DY) * numpy.fft.fft2(arr))))

[[-1  0  1]
 [ 0  0  0]
 [ 0  0  0]]
[[ 1.  1. -2.]
 [ 1.  1. -2.]
 [ 1.  1. -2.]]
